# Trabajar con entornos


Al ejecutar un script como un trabajo de Azure Machine Learning, debe definir el contexto de ejecución para la ejecución del trabajo. Una configuración de clave es el destino de proceso en el que se ejecutará el script. Podría tratarse de la estación de trabajo local (en este caso, la instancia de proceso) o un destino de proceso remoto, como el clúster de proceso administrado de Azure Machine Learning que se aprovisiona a petición.

En este cuaderno, creará un clúster de proceso y explorará los destinos de proceso para los trabajos.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Ejecución de un script como trabajo

Para entrenar un modelo, primero creará el script **diabetes_training.py** en la carpeta **src**. El script usa el archivo **diabetes.csv** en la misma carpeta que los datos de entrenamiento.

Tenga en cuenta que importa bibliotecas al principio del script. Las funciones de estas bibliotecas se usan para procesar los datos y entrenar el modelo. El proceso que use para ejecutar el script debe tener instaladas estas bibliotecas.

In [ ]:
Después de crear el script, puede ejecutar el script como un trabajo. El script usa bibliotecas comunes. Por lo tanto, puede usar un entorno mantenido que incluya pandas, numpy y scikit-learn, entre otros.

El trabajo usa la versión más reciente del entorno mantenido: `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`.

In [ ]:
Mientras se ejecuta el trabajo, ya puede ejecutar las celdas siguientes.

## Enumeración de entornos

Se explorarán los entornos del área de trabajo. 

En el trabajo anterior, usó uno de los entornos mantenidos. Para explorar todos los entornos que ya existen en el área de trabajo, puede enumerar los entornos: 

In [ ]:
Tenga en cuenta que todos los entornos mantenidos tienen nombres que comienzan por **AzureML(** (no puede usar este prefijo para sus propios entornos).

Para revisar un entorno específico, puede recuperar un entorno por su nombre y versión. Por ejemplo, puede recuperar la *descripción* y las *etiquetas* del entorno mantenido que usó para el trabajo anterior:

## Creación y uso de un entorno personalizado

Si un entorno mantenido no incluye todos los paquetes de Python que necesita para ejecutar el script, puede crear su propio entorno personalizado. Al enumerar todos los paquetes necesarios en un entorno, puede volver a ejecutar fácilmente los scripts. Todas las dependencias se almacenan en el entorno que puede especificar en la configuración del trabajo, independientemente del proceso que use.

Por ejemplo, puede crear un entorno simplemente a partir de una imagen de Docker. Algunos marcos como PyTorch tendrán una imagen pública de Docker que ya incluye todo lo que necesita. 

Se creará un entorno a partir de una imagen de Docker:

In [ ]:
El entorno ahora está registrado en el área de trabajo y puede hacer referencia a él al ejecutar un script como un trabajo:

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> El trabajo fallará rápidamente. Revise el mensaje de error. </p>

El mensaje de error le indicará que no hay ningún módulo denominado pandas. Hay dos posibles causas para este error:

- El script usa Pandas pero no importó la biblioteca (`import pandas as pd`). 
- El script importa la biblioteca en la parte superior del script, pero el proceso no tenía instalada la biblioteca (`pip install pandas`).

Después de revisar el script `diabetes-training.py`, puede observar que el script es correcto, lo que significa que la biblioteca no se instaló. En otras palabras, el entorno no incluía los paquetes necesarios.

Se creará un nuevo entorno con la imagen base de Docker usada en el trabajo anterior. Ahora, agregará una especificación de Conda para asegurarse de que se instalarán los paquetes necesarios. En primer lugar, ejecute la celda siguiente para crear el archivo de especificación de Conda:

Tenga en cuenta que todas las dependencias necesarias se incluyen en el archivo de especificación de Conda para que el script se ejecute correctamente.

Cree un entorno con la imagen base de Docker **y** el archivo de especificación de Conda para agregar las dependencias necesarias. Azure Machine Learning compila el entorno de Conda sobre la imagen de Docker proporcionada. 

In [ ]:
Ahora, puede enviar un trabajo con el nuevo entorno para ejecutar el script:

El envío del trabajo con el nuevo entorno personalizado desencadena la compilación del entorno. La primera vez que use un entorno recién creado, puede tardar entre 10 y 15 minutos en compilar el entorno, lo que también significa que el trabajo tardará más tiempo en completarse. 

También puede elegir desencadenar manualmente la compilación del entorno antes de enviar un trabajo. El entorno solo debe compilarse la primera vez que lo use. 

Cuando el trabajo desencadena la compilación de un nuevo entorno, puede revisar los registros de la compilación en la pestaña **Salidas y registros** del trabajo. Abra **azureml-logs/20_image_build_log.txt** para inspeccionar los registros de la compilación del entorno. 

![Captura de pantalla de los registros de compilación](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)